In [1]:
from pyspark.sql import SparkSession, types, functions as F
import datetime

In [2]:
spark = SparkSession.builder\
          .master('local[*]')\
          .appName('test')\
          .getOrCreate()

25/03/31 15:23:42 WARN Utils: Your hostname, Bastiens-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 10.243.218.61 instead (on interface en0)
25/03/31 15:23:42 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/31 15:23:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Measurements

In [12]:
def get_datetime(day, s):
  delta = datetime.timedelta(seconds=s)
  return day + delta

get_datetime_udf = F.udf(get_datetime, returnType=types.TimestampType())

In [13]:
def get_datetime_hour(d):
  return d.hour

get_datetime_hour_udf = F.udf(get_datetime_hour, returnType=types.IntegerType())

In [14]:
measurements_schema = types.StructType([
  types.StructField('day', types.TimestampType(), False),
  types.StructField('interval', types.IntegerType(), False),
  types.StructField('detid', types.StringType(), False),
  types.StructField('flow', types.IntegerType(), True),
  types.StructField('occ', types.FloatType(), True),
  types.StructField('error', types.FloatType(), True),
  types.StructField('city', types.StringType(), False),
  types.StructField('speed', types.FloatType(), True)
])

In [15]:
measurements_df = spark.read\
                        .option('header', True)\
                        .schema(measurements_schema)\
                        .csv('./data/measurements_test.csv')

In [16]:
measurements_df = measurements_df\
  .withColumn('error', F.col('error').cast('integer'))\
  .withColumn('datetime', get_datetime_udf('day', 'interval'))\
  .withColumn("date", F.to_date("day"))\
  .withColumn('hour', get_datetime_hour_udf('datetime'))\
  .drop('interval', 'datetime', 'day')

In [8]:
measurements_df.printSchema()

root
 |-- detid: string (nullable = true)
 |-- flow: integer (nullable = true)
 |-- occ: float (nullable = true)
 |-- error: integer (nullable = true)
 |-- city: string (nullable = true)
 |-- speed: float (nullable = true)
 |-- date: date (nullable = true)
 |-- hour: integer (nullable = true)



In [14]:
agg_measurements_df = measurements_df \
                    .groupBy(['city', 'detid', 'date', 'hour'])\
                    .agg({
                      "flow": "avg",
                      "occ": "avg",
                      "speed": "avg",
                      "error": "array_agg"
                    })

In [15]:
agg_measurements_df.show(10)

+--------+--------+----------+----+--------------------+---------+--------------------+----------+
|    city|   detid|      date|hour| collect_list(error)|avg(flow)|            avg(occ)|avg(speed)|
+--------+--------+----------+----+--------------------+---------+--------------------+----------+
|augsburg|06.X-2li|2017-05-06|   0|[1, 1, 1, 1, 1, 1...|     14.0|0.002499999944120...|      NULL|
|augsburg|06.X-2li|2017-05-06|   1|[1, 1, 1, 1, 1, 1...|      4.0|                 0.0|      NULL|
|augsburg|06.X-2li|2017-05-06|   2|[1, 1, 1, 1, 1, 1...|      7.0|                 0.0|      NULL|
|augsburg|06.X-2li|2017-05-06|   3|[1, 1, 1, 1, 1, 1...|      6.0|                 0.0|      NULL|
|augsburg|06.X-2li|2017-05-06|   4|[1, 1, 1, 1, 1, 1...|      5.0|                 0.0|      NULL|
|augsburg|06.X-2li|2017-05-06|   5|[1, 1, 1, 1, 1, 1...|     14.0| 0.02500000037252903|      NULL|
|augsburg|06.X-2li|2017-05-06|   6|[1, 1, 1, 1, 1, 1...|     13.0|0.012500000186264515|      NULL|
|augsburg|

In [9]:
measurements_df.show(20)

+--------+----+----+-----+--------+-----+----------+----+
|   detid|flow| occ|error|    city|speed|      date|hour|
+--------+----+----+-----+--------+-----+----------+----+
|06.X-2li|  12| 0.0|    1|augsburg| NULL|2017-05-06|   0|
|06.X-2li|  12| 0.0|    1|augsburg| NULL|2017-05-06|   0|
|06.X-2li|  12| 0.0|    1|augsburg| NULL|2017-05-06|   0|
|06.X-2li|  16| 0.0|    1|augsburg| NULL|2017-05-06|   0|
|06.X-2li|  16| 0.0|    1|augsburg| NULL|2017-05-06|   0|
|06.X-2li|  16| 0.0|    1|augsburg| NULL|2017-05-06|   0|
|06.X-2li|  20| 0.0|    1|augsburg| NULL|2017-05-06|   0|
|06.X-2li|  20| 0.0|    1|augsburg| NULL|2017-05-06|   0|
|06.X-2li|  20| 0.0|    1|augsburg| NULL|2017-05-06|   0|
|06.X-2li|   8|0.01|    1|augsburg| NULL|2017-05-06|   0|
|06.X-2li|   8|0.01|    1|augsburg| NULL|2017-05-06|   0|
|06.X-2li|   8|0.01|    1|augsburg| NULL|2017-05-06|   0|
|06.X-2li|   4| 0.0|    1|augsburg| NULL|2017-05-06|   1|
|06.X-2li|   4| 0.0|    1|augsburg| NULL|2017-05-06|   1|
|06.X-2li|   4

## Detectors

In [3]:
detectors_schema = types.StructType([
  types.StructField('detid', types.StringType(), False),
  types.StructField('length', types.DoubleType(), False),
  types.StructField('pos', types.DoubleType(), False),
  types.StructField('fclass', types.StringType(), False),
  types.StructField('road', types.StringType(), False),
  types.StructField('limit', types.IntegerType(), False),
  types.StructField('citycode', types.StringType(), False),
  types.StructField('lanes', types.IntegerType(), False),
  types.StructField('linkid', types.IntegerType(), False),
  types.StructField('long', types.DoubleType(), False),
  types.StructField('lat', types.DoubleType(), False)
])

In [4]:
detectors_df = spark.read\
                .option("header", True)\
                .schema(detectors_schema)\
                .csv('./data/detectors_public.csv')

In [5]:
detectors_df.printSchema()

root
 |-- detid: string (nullable = true)
 |-- length: double (nullable = true)
 |-- pos: double (nullable = true)
 |-- fclass: string (nullable = true)
 |-- road: string (nullable = true)
 |-- limit: integer (nullable = true)
 |-- citycode: string (nullable = true)
 |-- lanes: integer (nullable = true)
 |-- linkid: integer (nullable = true)
 |-- long: double (nullable = true)
 |-- lat: double (nullable = true)



## Links

In [7]:
links_schema = types.StructType([
  types.StructField('long', types.DoubleType(), False),
  types.StructField('lat', types.DoubleType(), False),
  types.StructField('order', types.IntegerType(), False),
  types.StructField('piece', types.IntegerType(), False),
  types.StructField('linkid', types.IntegerType(), False),
  types.StructField('group', types.FloatType(), False),
  types.StructField('citycode', types.StringType(), False)
])

In [8]:
links_df = spark.read\
            .option("header", True)\
            .schema(links_schema)\
            .csv('./data/links.csv')

In [9]:
links_df.printSchema()

root
 |-- long: double (nullable = true)
 |-- lat: double (nullable = true)
 |-- order: integer (nullable = true)
 |-- piece: integer (nullable = true)
 |-- linkid: integer (nullable = true)
 |-- group: float (nullable = true)
 |-- citycode: string (nullable = true)



In [10]:
links_df.show(30)

+----------+----------+-----+-----+------+-----+--------+
|      long|       lat|order|piece|linkid|group|citycode|
+----------+----------+-----+-----+------+-----+--------+
|10.8910158|48.3610789|    1|    1|     0|  0.1|augsburg|
|10.8908938|48.3609933|    2|    1|     0|  0.1|augsburg|
|10.8906417|48.3608526|    3|    1|     0|  0.1|augsburg|
|10.8904043|48.3607189|    4|    1|     0|  0.1|augsburg|
|10.8899939| 48.360497|    5|    1|     0|  0.1|augsburg|
|10.8909796|48.3611074|    1|    1|     1|  1.1|augsburg|
|10.8908589|48.3610201|    2|    1|     1|  1.1|augsburg|
| 10.890588|48.3608561|    3|    1|     1|  1.1|augsburg|
|10.8903842|48.3607421|    4|    1|     1|  1.1|augsburg|
|10.8901803|48.3606378|    5|    1|     1|  1.1|augsburg|
|10.8899577|48.3605166|    6|    1|     1|  1.1|augsburg|
|10.8935881|48.3648605|    1|    1|     2|  2.1|augsburg|
|10.8935827| 48.364693|    2|    1|     2|  2.1|augsburg|
|10.8934379|48.3644453|    3|    1|     2|  2.1|augsburg|
|10.8928666|48

## Merge

In [22]:
df = measurements_df.join(
  F.broadcast(detectors_df),
  on=['detid'],
  how='left',
)

In [23]:
df.show(10)

+--------+----+----+-----+--------+-----+----------+----+-----------------+-----------------+--------+--------------------+-----+--------+-----+------+----------+----------+
|   detid|flow| occ|error|    city|speed|      date|hour|           length|              pos|  fclass|                road|limit|citycode|lanes|linkid|      long|       lat|
+--------+----+----+-----+--------+-----+----------+----+-----------------+-----------------+--------+--------------------+-----+--------+-----+------+----------+----------+
|06.X-2li|  12| 0.0|    1|augsburg| NULL|2017-05-06|   0|0.112002945639535|0.003422816566385|tertiary|Reichenberger Straße|   30|augsburg|    1|   737|10.9294921|48.3636549|
|06.X-2li|  12| 0.0|    1|augsburg| NULL|2017-05-06|   0|0.112002945639535|0.003422816566385|tertiary|Reichenberger Straße|   30|augsburg|    1|   737|10.9294921|48.3636549|
|06.X-2li|  12| 0.0|    1|augsburg| NULL|2017-05-06|   0|0.112002945639535|0.003422816566385|tertiary|Reichenberger Straße|   30|a

In [ ]:
spark.stop()